In [2]:
from operator import index
import streamlit as st
import plotly.express as px
from pycaret.regression import setup, compare_models, pull, save_model, load_model
import pandas_profiling
import pandas as pd
from streamlit_pandas_profiling import st_profile_report
import os,re
import trading_data,trading_lstm


from binance.spot import Spot 
from sys import stderr
import sys, os,datetime,requests,json,pandas as pd,numpy as np
import time,math, gc

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import sklearn
from sklearn.model_selection import train_test_split
min_max_scaler = sklearn.preprocessing.MinMaxScaler()

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision



2023-05-09 13:19:46.845 INFO    visions.backends: Pandas backend loaded 1.5.2
2023-05-09 13:19:46.855 INFO    visions.backends: Numpy backend loaded 1.23.5
2023-05-09 13:19:46.857 INFO    visions.backends: Pyspark backend NOT loaded
2023-05-09 13:19:46.858 INFO    visions.backends: Python backend loaded


In [5]:
os.path.splitext(r"aaaa/aaaa/exc.txt")[1][1:]

'txt'

In [2]:
directory="streamlit_data"
selected_folder=directory
currency_name="ETHUSDT"

with st.sidebar: 
    st.image("https://www.onepointltd.com/wp-content/uploads/2020/03/inno2.png")
    #st.title("AutoNickML")
    choice = st.radio("Navigation", ["Choose currency","Bot training","Data description","Bot using"])
    #st.info("This project application helps you build and explore your data.")
    
if choice=="Choose currency":
    st.title("Choosing and downloading currency data")
    currency_name=st.text_input(label="Input chosen currency pair here. Example: ETH/USDT")
    
    if (currency_name):
        currency_name=re.sub('[^0-9a-zA-Z]+', '', currency_name)
        days_downloading=st.text_input(label="Set how many days you want the bot to train on")
        if (days_downloading):
            try:
                days_downloading=int(days_downloading)
                selected_folder = st.text_input(label="Input chosen folder path or leave empty to set default path")
                if (selected_folder==""):
                    selected_folder=directory
                downloaded_data=trading_data.download_klines(currency_name,selected_folder,days_downloading,target_count=4)
                st.text("Your data has been downloaded successfully!")
                st.dataframe(downloaded_data.head(5))
                displayed_chart=downloaded_data["Open"][-3000:].plot().get_figure()
                displayed_chart.savefig(os.path.join(directory,f"{currency_name}_chart.jpg"))
                st.image(os.path.join(directory,f"{currency_name}_chart.jpg"))
            except:
                print("error")
                raise

if choice=="Bot training":
    scaling=1
    st.title("Uploading the data to train the bot on")
    file=st.file_uploader(label="Upload the data")
    if (file):
        X_train, X_test, y_train, y_test, X_train_opens, X_test_opens=trading_lstm.prepare_data_for_train(file)
        net,net_min_err_train,net_min_err_test,net_max_std,err_df,stds_single=trading_lstm.train_bot( X_train, X_test, y_train, y_test,X_train_opens, X_test_opens,epochs=100)
        model_scripted = torch.jit.script(net_min_err_test) # Export to TorchScript
        model_scripted.save('model_scripted.pt') # Save
        displayed_chart_train=err_df.plot().get_figure()
        displayed_chart_train.savefig(os.path.join(directory,"bot_losses_chart.jpg"))
        st.image(os.path.join(directory,"bot_losses_chart.jpg"))
        
        
if choice=="Bot using":
    model=st.file_uploader(label="Upload the data")
    if (model):
        trading_bot.run_bot(symbol="ETHUSDT",directory="bot_data", net_file=model)

if choice=="Data description":
    file=st.file_uploader(label="Upload the data")
    if (file):
        df=pd.read_csv(file).reset_index(drop=True)
        st_profile_report(df.profile_report())

2023-01-10 14:37:11.234 
  command:

    streamlit run G:\anaconda3\envs\trading_streamlit\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
